In [1]:
using BenchmarkTools, Logging, Revise
global_logger(ConsoleLogger(stdout, Logging.Info));

In [2]:
using PorousMaterials, MOFun

[ Info: Precompiling MOFun [top-level]


In [3]:
s_moty = moiety("find-replace/2-!-p-phenylene")
r_moty = moiety("2-acetylamido-p-phenylene")
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)
search = s_moty ∈ parent;

# single random replacement w/o search kwarg

In [4]:
@btime find_replace(s_moty, r_moty, parent);

  2.444 s (13347700 allocations: 3.58 GiB)


# single random replacement w/ search kwarg

In [5]:
@btime find_replace(s_moty, r_moty, parent, search=search);

  931.200 μs (23604 allocations: 1.60 MiB)


# single specific replacement

In [6]:
@btime find_replace(s_moty, r_moty, parent, search=search, config=Configuration(1,1));

  933.100 μs (23605 allocations: 1.60 MiB)


# list of (all) replacements
## this runs 40,000-fold faster after refactoring!
### I could functionalize 70,000 MOFs with 20 R-groups in a day on one HPC node!!
#### note to self: write a distributed version...

In [7]:
configs = [Configuration(loc, rand(1:search.num_orientations[loc])) for loc in 1:search.num_locations]
@btime find_replace(s_moty, r_moty, parent, search=search, configs=configs);

  2.930 ms (58125 allocations: 3.63 MiB)


# save output

In [8]:
xtal = find_replace(s_moty, r_moty, parent, search=search, configs=configs)
write_cif(xtal, "OUTPUT")